In [1]:
import torch
from torch.utils.data import DataLoader
from miditok import REMI
from miditok.pytorch_data import DatasetMIDI, DataCollator
from pathlib import Path
import json
import pandas as pd
import numpy as np
import random

In [2]:
import features_vectors_2 as utils

In [3]:
import importlib

importlib.reload(utils)

<module 'features_vectors_2' from '/mnt/nfs_share_magnet1/lafuente/symbolic_music/author-profiling/experiments/e15/features_vectors_2.py'>

In [4]:
k=1
train_data=pd.read_csv(f'../../train data/k folds/type0_train_set_k{k}.csv')
validation_data=pd.read_csv(f'../../train data/k folds/type0_test_set_k{k}.csv')

In [5]:
val_dataloader=utils.create_data_loader(scores_df=validation_data,paths_column_name='type0_path')
train_dataloader=utils.create_data_loader(scores_df=train_data,paths_column_name='type0_path')

/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/classes.py:702: UserWarning: Argument nb_tempos has been renamed num_tempos, you should consider to updateyour code with this new argument name.
  return cls(**input_dict, **kwargs)


In [6]:
train_feature_vectors=utils.get_feature_vectors(dataloader=train_dataloader,
                                                dataframe=train_data,
                                                set_type='train', 
                                                feature_tensors=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
computing feature tensors: 100%|██████████████| 184/184 [08:27<00:00,  2.76s/it]


last hidden state list shape torch.Size([5110, 512])


In [ ]:
val_feature_vectors=utils.get_feature_vectors(dataloader=val_dataloader,
                                                dataframe=validation_data,
                                                set_type='val', 
                                                feature_tensors=False)

computing feature tensors:  67%|██████████▊     | 31/46 [01:19<00:37,  2.53s/it]

In [ ]:
from torch.utils.data import Dataset,DataLoader

seed = 42
if seed is not None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
class DatasetMLP(Dataset):

    def __init__(self,data):
        self.data=data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self,ind):
        x=self.data[ind][:-1]
        y=self.data[ind][-1]

        return x,y

class TestDataset(DatasetMLP):
    def __getitem__(self,ind):
        x=self.data[ind]
        return x

train_set_mlp=DatasetMLP(np.array(train_feature_vectors))
val_set_mlp=DatasetMLP(np.array(val_feature_vectors))

batch_size=20

train_dataloder_mlp=DataLoader(train_set_mlp,
                               batch_size=batch_size,
                               shuffle=True)  

val_dataloder_mlp=DataLoader(val_set_mlp,
                               batch_size=batch_size,
                               shuffle=False)  

In [ ]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#class MLP(nn.Module):
#    def __init__(self, input_dim):
#        super(MLP, self).__init__()
#        self.linear = nn.Linear(input_dim, 2)  # Output 2 classes
    
#    def forward(self, x):
#        out = self.linear(x)
#        return out

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1=128, hidden_dim2=64, hidden_dim3=32, output_dim=2):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = nn.Linear(hidden_dim3, output_dim)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        out = self.fc4(x)
        return out

from sklearn.metrics import balanced_accuracy_score

def evaluate(model, dataloader, criterion):
    model.eval()  # Set model to evaluation mode
    predictions = []
    true_labels = []
    losses = []

    with torch.no_grad():
        for input_data in dataloader:
            x, y = input_data
            x = x.to(device).float()
            y = y.to(device).long()

            output = model(x)
            _, predicted = torch.max(output, 1)

            batch_predictions = predicted.cpu().detach().numpy().tolist()
            batch_true_labels = y.cpu().detach().numpy().tolist()

            predictions.extend(batch_predictions)
            true_labels.extend(batch_true_labels)

            # Compute loss
            loss = criterion(output, y)
            losses.append(loss.item())

    # Compute average validation loss
    avg_loss = sum(losses) / len(losses)

    # Compute balanced accuracy
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    balanced_accuracy = balanced_accuracy_score(true_labels, predictions)

    return balanced_accuracy, avg_loss, predictions, true_labels


In [ ]:
from sklearn.metrics import balanced_accuracy_score
import torch.optim.lr_scheduler as lr_scheduler



seed = 42
if seed is not None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
input_dim = 51100
model = MLP(input_dim).to(device)

#initial_lr = 0.001
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
# Assuming optimizer is already defined
#scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

print(model)

epochs = 10

model.train()
train_avg_loss_list=[]
val_avg_loss_list=[]
train_balanced_accuracy_list=[]
val_balanced_accuracy_list=[]

for epoch in range(epochs):
    losses = []
    predictions = []
    true_labels = []
    for batch_num, input_data in enumerate(train_dataloder_mlp):
        optimizer.zero_grad()
        x, y = input_data
        x = x.to(device).float()
        y = y.to(device).long()  # Ensure y is of type long for CrossEntropyLoss

        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        losses.append(loss.item())

        optimizer.step()

        # Convert predictions to class labels (0 or 1)
        _, predicted = torch.max(output, 1)
        batch_predictions = predicted.cpu().detach().numpy().tolist()
        batch_true_labels = y.cpu().detach().numpy().tolist()

        predictions.extend(batch_predictions)
        true_labels.extend(batch_true_labels)

        #if batch_num % 40 == 0:
        #    print('\tEpoch %d | Batch %d | Loss %6.2f' % (epoch, batch_num, loss.item()))
    # Step the scheduler
    #scheduler.step()
    
    train_balanced_accuracy = balanced_accuracy_score(true_labels, predictions)
    train_avg_loss=sum(losses)/len(losses) 
    print('Epoch %d | Train Loss %6.2f| Train Balanced Accuracy %6.2f' % (epoch, train_avg_loss ,train_balanced_accuracy))
    
    val_balanced_accuracy, val_avg_loss, val_predictions, val_true_labels = evaluate(model, val_dataloder_mlp,criterion)
    print('Epoch %d | Validation Loss %6.2f| Validation Balanced Accuracy: %6.2f' % (epoch, val_avg_loss, val_balanced_accuracy))

    train_avg_loss_list.append(train_avg_loss)
    val_avg_loss_list.append(val_avg_loss)
    
    train_balanced_accuracy_list.append(train_balanced_accuracy)
    val_balanced_accuracy_list.append(val_balanced_accuracy)


# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Example of using predictions and true labels
print("Predictions:", predictions)
print("True Labels:", true_labels)
